# Species tree distance matrix

We need to make a distance matrix from a Newick format tree with branch lengths.
This is a single call in R, but I'm kind of tired of switching back and forth between
the two, and this should be a fairly straightforward task.

This *was* extremely slow using the naive algorithm, then I got Wandrille's
improved method, which works great.

In [ ]:
import ete3
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
# Let's load in the tree.

tree = ete3.Tree("output/cleaned_trees/Actinopterygii_species_with_order.nwk", format=1, quoted_node_names=True)

In [ ]:
# Let's make sure there are no duplicated names in the tree. Check all tips - the internal nodes are not important for this.
# Print the list of duplicated names, if any.
tip_names = [leaf.name for leaf in tree.iter_leaves()]
duplicated_names = set([name for name in tip_names if tip_names.count(name) > 1])
print("Duplicated names:", duplicated_names)


Here is some code from Wandrille that runs a lot faster, however uses a bit of
RAM. (As a benchmark, takes about 5 minutes on Precision 7750 with 128GB RAM to
compute the distance matrix for ~15,000 species, aboout 2.5 minutes on an
Alienware Area 51 with core i9 and 64GB RAM). Although this seems to be variable
depending on the cruft/etc in RAM - a run on a fresh kernel on the same
Alienware took just under 30 seconds.

In [ ]:
## alternatively, to save memory: write the distances to file as they are computed
leaf_to_pos = {l:i for i,l in enumerate( tree.get_leaf_names() )}
DMat = np.zeros((len(leaf_to_pos),len(leaf_to_pos)))

for n in tree.traverse('postorder'):
    
    if n.is_leaf():
        n.dist_dict = {n.name : 0.0}
    else:
        
        n.dist_dict = {}
        
        ## populate distance dictionary
        for c in n.children:
            for l,d in c.dist_dict.items():
                n.dist_dict[l] = d + c.dist # adding the child's branch length
            
        # Now we combine the leaves of each pair of children to get pairwise distances:
        for i,c1 in enumerate( n.children ):
            for j,c2 in enumerate( n.children ):
                if j <= i :
                    continue
                
                ## for each pair of leaves
                for l1 in c1.dist_dict.keys():
                    for l2 in c2.dist_dict.keys():
                        ## important: grab the distance from the current node dict (to account for children branch lengths)
                        d = n.dist_dict[l1] + n.dist_dict[l2]
                        DMat[ leaf_to_pos[l1] , leaf_to_pos[l2] ] = d
                        DMat[ leaf_to_pos[l2] , leaf_to_pos[l1] ] = d
                

        # Lastly, we remove the children's dictonaries (no need to keep them)
        for c in n.children:
            c.dist_dict = None

    
python_dist_matrix = pd.DataFrame( DMat , index=tree.get_leaf_names(), columns=tree.get_leaf_names() )


In [ ]:
# Save
python_dist_matrix.to_csv("output/Actinopterygii_species_distance_matrix_py.csv")

# Deallocate DMat
DMat = None

# (OPTIONAL) Some statistics on distances

The following cells calculate some stats on the distance matrix - a histogram of distances, etc. These use a lot of RAM and can result in a very large notebook file, which can be problematic to load in Windows VS Code for some reason. Seems to work in the WSL version, which really makes no sense. Whatever.

In the end, <i>don't run these unless you need to</i>.

Doing statistics in the R notebook is far preferable - it's set up to do these kinds of things much faster.

In [ ]:
# What's the distribution of distances? Are there a lot of them between 0 and 10?
# Let's make a histogram that shows the distribution of distances in the
# distance matrix.

# First we need to make a list of all the distances. The distance matrix is a
# diagonal matrix, so we do not need each and every value, just the ones in the
# upper triangle.
python_distances = python_dist_matrix.where(np.triu(np.ones(python_dist_matrix.shape), k=1).astype(bool))

python_distances = python_distances.stack().reset_index()

python_distances.columns = ["taxon1", "taxon2", "distance"]

In [ ]:

# Now a histogram of all the distances.

px.fig = px.histogram(python_distances, x="distance", nbins=50, title="Histogram of Distances")
px.fig.show()

In [ ]:
python_distances = None
python_log_distances = None